<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/GSE135922_GSM4037981_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE135922_GSM4037981

This notebook uses the filtered count matrix resulting from the following google colab notebook. https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE135922_GSM4037981.ipynb Please run the notebook above and upload the results file in /content

In [1]:
# Install SRA-toolkit 
!wget "http://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz"

! tar -xzf sratoolkit.current-centos_linux64.tar.gz

# Add to path
import os
os.environ['PATH'] += ":/content/sratoolkit.2.11.0-centos_linux64/bin"

# Configure
!vdb-config --interactive

# Import packages

import numpy as np


--2021-04-12 17:48:45--  http://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.11, 2607:f220:41e:250::11, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz [following]
--2021-04-12 17:48:46--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99139357 (95M) [application/x-gzip]
Saving to: ‘sratoolkit.current-centos_linux64.tar.gz’

sratoolkit.current- 100%[===================>]  94.55M   354MB/s    in 0.3s    

2021-04-12 17:48:46 (354 MB/s) - 

In [2]:
# List of SRAs to download and transform. Copy paste from excel as string
SRAs = "SRR9990627	SRR9990628	SRR9990629	SRR9990630	SRR9990631	SRR9990632	SRR9990633	SRR9990634	SRR9990635	SRR9990636	SRR9990637	SRR9990638	SRR9990639	SRR9990640	SRR9990641	SRR9990642	SRR9990643	SRR9990644	SRR9990645	SRR9990646	SRR9990647	SRR9990648	SRR9990649	SRR9990650"

SRAs = SRAs.split()

# SRA to delete, usually the index file, which get downloaded because we include technical reads. 
# It varies so check in the SRA website
SRA_to_delete = "3"


SRAs_to_keep = np.setdiff1d(["1", "2", "3"], [SRA_to_delete])

# sample specific metadata

samp_id = ["GSM4037981"]

cell_type = ["retinal pigment epithelium macula, unenriched"]

condition = ["donor 1"]

In [3]:
for sra in SRAs:
  """
  Downloads SRAs and transforms to compressed fastq file. Remove index reads and original SRA
  """
  # Download SRA
  !prefetch $sra

  # Convert to fastq. Biological reads are usually considered technicals in 10x so do not skip them.
  fastq_dump_cmd = "fastq-dump -F --gzip --readids --split-files " + sra + "/" + sra +".sra"
  !$fastq_dump_cmd

  # Remove files
  rm_cmd_1 = "rm " + sra + "/" + sra + ".sra"
  rm_cmd_2 = "rm " + sra + "_" + SRA_to_delete + ".fastq.gz" 
  !$rm_cmd_1
  !$rm_cmd_2


2021-04-12T17:49:54 prefetch.2.11.0: 1) Downloading 'SRR9990627'...
2021-04-12T17:49:54 prefetch.2.11.0:  Downloading via HTTPS...
2021-04-12T17:50:28 prefetch.2.11.0:  HTTPS download succeed
2021-04-12T17:50:31 prefetch.2.11.0:  'SRR9990627' is valid
2021-04-12T17:50:31 prefetch.2.11.0: 1) 'SRR9990627' was downloaded successfully
2021-04-12T17:50:31 prefetch.2.11.0: 'SRR9990627' has 0 unresolved dependencies
Read 10536649 spots for SRR9990627/SRR9990627.sra
Written 10536649 spots for SRR9990627/SRR9990627.sra
rm: cannot remove 'SRR9990627_3.fastq.gz': No such file or directory

2021-04-12T18:07:28 prefetch.2.11.0: 1) Downloading 'SRR9990628'...
2021-04-12T18:07:28 prefetch.2.11.0:  Downloading via HTTPS...
2021-04-12T18:08:04 prefetch.2.11.0:  HTTPS download succeed
2021-04-12T18:08:07 prefetch.2.11.0:  'SRR9990628' is valid
2021-04-12T18:08:07 prefetch.2.11.0: 1) 'SRR9990628' was downloaded successfully
2021-04-12T18:08:07 prefetch.2.11.0: 'SRR9990628' has 0 unresolved dependencies


In [4]:
# Define fastq list to feed kb

fastqs = []
for sra in SRAs:
  for read in SRAs_to_keep:
    fastqs.append(sra + "_" + read + ".fastq.gz")

## Metadata

In [5]:
# define the values for the analysis

# accession id for the data
id = "GSE135922"

no_samples = 1

fastqs_per_sample = [8] 

sample_id = samp_id

database_id = [id] * no_samples

tissue = ["eye"] * no_samples

species = ["human"] * no_samples

technology = ["10xv3"] * no_samples

paper = ["Sungnak et al 2020"] * no_samples

figure = ["Fig 1"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]

# Imports and installs

In [6]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 59.1MB 107kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 133kB 43.7MB/s 
     |████████████████████████████████| 10.3MB 25.2MB/s 
     |████████████████████████████████| 13.2MB 182kB/s 
     |████████████████████████████████| 112kB 49.5MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 1.2MB 41.2MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 


# Downloads: index

In [ ]:
# Download the corresponding Kallisto index to fastq folder
!kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

# Process fastq files (modify kb command according to fastqs list)


In [ ]:
fastqs

In [ ]:
if no_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
  + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(no_samples):

    fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
    # Write the kb count command as a string
    cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
    --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

    # Execute it
    !$cmd

    # Update j to move to the next set of fastq
    j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd


import time
time.sleep(60000000)

[2021-04-13 02:45:00,945]    INFO Using index index.idx to generate BUS file to tccoutputGSM4037981 from
[2021-04-13 02:45:00,945]    INFO         SRR9990627_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990627_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990628_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990628_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990629_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990629_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990630_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990630_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990631_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990631_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990632_1.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990632_2.fastq.gz
[2021-04-13 02:45:00,945]    INFO         SRR9990633_1.fastq.gz
[2021-04-13 02:45:00,946]    INFO         SRR9990633_2.fastq.gz

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


  adding: bus_raw_GSM4037981_from_gc/ (stored 0%)
  adding: bus_raw_GSM4037981_from_gc/tcc_unfiltered/ (stored 0%)
  adding: bus_raw_GSM4037981_from_gc/tcc_unfiltered/adata.h5ad.gz (deflated 8%)
  adding: bus_raw_GSM4037981_from_gc/tcc_unfiltered/cells_x_tcc.mtx.gz (deflated 0%)
  adding: bus_raw_GSM4037981_from_gc/tcc_unfiltered/cells_x_tcc.barcodes.txt.gz (deflated 1%)
  adding: bus_raw_GSM4037981_from_gc/tcc_unfiltered/cells_x_tcc.ec.txt.gz (deflated 2%)
  adding: bus_raw_GSM4037981_from_gc/inspect.json (deflated 57%)
  adding: bus_raw_GSM4037981_from_gc/GSM4037981.unfiltered.bus.gz (deflated 3%)
  adding: bus_raw_GSM4037981_from_gc/run_info.json (deflated 73%)
  adding: bus_raw_GSM4037981_from_gc/kb_info.json (deflated 82%)
